In [40]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import Dataset,DataLoader
import numpy as np
import math

In [41]:
class WineDataset(Dataset):
    
    def __init__(self):
        data=np.loadtxt('./data/wine.csv',delimiter=",",dtype=np.float32,skiprows=1) #skip header
        self.x = torch.from_numpy(data[: , 1:])
        self.y = torch.from_numpy(data[:, 0])
        self.y=torch.tensor(self.y,dtype=torch.int64)
        self.n_samples = data.shape[0] 
    def __getitem__(self,idx):
        return self.x[idx],self.y[idx]
    
    def __len__(self):
        return self.n_samples
    
    
    

In [42]:
class Network(nn.Module):
    
    def __init__(self,input_dim,output_dim,lr,l1_size,l2_size):
        
        super(Network, self).__init__()
        self.input_dim=input_dim
        self.output_dim=output_dim
        self.lr=lr
        self.l1_size=l1_size
        self.l2_size=l2_size
#         self.optimizer=torch.optim.Adam(self.parameters(),lr=self.lr)
        # self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        self.l1=nn.Linear(input_dim,self.l1_size)
        self.l2=nn.Linear(self.l1_size,self.l2_size)
        self.l3=nn.Linear(self.l2_size,output_dim)
        
    def forward(self,x):
        # x=x.to(self.device)
        x=F.relu(self.l1(x))
        x=F.relu(self.l2(x))
        x=self.l3(x)
        # output=torch.softmax(x,dim=1)
        
        return x
        
        

In [43]:
n=Network(13,3,0.001,8,8)
optimizer=torch.optim.Adam(n.parameters(),lr=n.lr)
criterion=nn.CrossEntropyLoss()

In [61]:
dataset = WineDataset()
train_set,test_set,val_set=torch.utils.data.random_split(dataset,[142,30,6])
# dataloader=DataLoader(dataset=dataset, batch_size=8, shuffle=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys


In [45]:
train_set=DataLoader(dataset=train_set,batch_size=4,shuffle=True)
example=iter(train_set)
sample,label=example.next()
print(sample.shape,label-1)

torch.Size([4, 13]) tensor([2, 2, 1, 2])


In [46]:
for e in range(1000):
    running_loss=0
    for i,(inputs,labels) in enumerate(train_set):
        outputs=n(inputs)
        labels=labels-1
        loss=criterion(outputs,labels)
        running_loss+=loss
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print("Epoch:"+str(e)+" Loss:"+str(running_loss/len(train_set)))

Epoch:0 Loss:tensor(8.4139, grad_fn=<DivBackward0>)
Epoch:1 Loss:tensor(1.0322, grad_fn=<DivBackward0>)
Epoch:2 Loss:tensor(0.8745, grad_fn=<DivBackward0>)
Epoch:3 Loss:tensor(0.8627, grad_fn=<DivBackward0>)
Epoch:4 Loss:tensor(0.8616, grad_fn=<DivBackward0>)
Epoch:5 Loss:tensor(0.8405, grad_fn=<DivBackward0>)
Epoch:6 Loss:tensor(0.8369, grad_fn=<DivBackward0>)
Epoch:7 Loss:tensor(0.8153, grad_fn=<DivBackward0>)
Epoch:8 Loss:tensor(0.8029, grad_fn=<DivBackward0>)
Epoch:9 Loss:tensor(0.8053, grad_fn=<DivBackward0>)
Epoch:10 Loss:tensor(0.7949, grad_fn=<DivBackward0>)
Epoch:11 Loss:tensor(0.7944, grad_fn=<DivBackward0>)
Epoch:12 Loss:tensor(0.7774, grad_fn=<DivBackward0>)
Epoch:13 Loss:tensor(0.7746, grad_fn=<DivBackward0>)
Epoch:14 Loss:tensor(0.7715, grad_fn=<DivBackward0>)
Epoch:15 Loss:tensor(0.7606, grad_fn=<DivBackward0>)
Epoch:16 Loss:tensor(0.7527, grad_fn=<DivBackward0>)
Epoch:17 Loss:tensor(0.7586, grad_fn=<DivBackward0>)
Epoch:18 Loss:tensor(0.7527, grad_fn=<DivBackward0>)
Epo

In [62]:
#testing
test_set=DataLoader(dataset=test_set,batch_size=4,shuffle=True)


In [64]:
with torch.no_grad():
  n_correct=0
  n_samples=0

  for i,(inputs,labels) in enumerate(test_set):
      outputs=n(inputs)
      labels=labels-1
      
      _,predictions=torch.max(outputs,1)
      n_samples+=labels.shape[0]
      print(predictions,labels)
      n_correct+=(predictions==labels).sum().item()
  acc=100.0*n_correct/n_samples
  print("Accuracy:",acc)

tensor([0, 0, 1, 2]) tensor([0, 0, 1, 2])
tensor([1, 0, 1, 1]) tensor([1, 0, 1, 1])
tensor([1, 1, 0, 0]) tensor([1, 1, 0, 0])
tensor([1, 2, 2, 2]) tensor([1, 2, 2, 2])
tensor([0, 1, 0, 2]) tensor([0, 1, 0, 2])
tensor([0, 1, 1, 1]) tensor([0, 1, 1, 1])
tensor([0, 1, 1, 0]) tensor([0, 1, 0, 0])
tensor([0, 1]) tensor([0, 2])
Accuracy: 93.33333333333333
